### Introduction

|CO(GT)|Feature|Integer|True hourly averaged concentration CO in mg/m^3 (reference analyzer) | mg/m^3 |	no

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Read Data Set

In [2]:
df = pd.read_csv(r"C:\Users\ryana\Downloads\DataSet\AirQualityUCI.csv", sep=';')
pd.set_option('display.max_columns', None)

In [3]:
df

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578",NaN,NaN
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255",NaN,NaN
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502",NaN,NaN
3,10/03/2004,21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867",NaN,NaN
4,10/03/2004,22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Summary

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.duplicated().sum()

In [ ]:
df.shape

### Cleaning Data

#### Drop Unused Column

In [ ]:
df = df.drop(columns=['Unnamed: 15', 'Unnamed: 16'], axis=1)

#### Filter Exclude the NaN Value

In [ ]:
df = df[df['Date'].notna()]

#### Noticed that duplicated Value has been Eliminated

In [ ]:
df.duplicated().sum()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

#### Change Data Type

In [ ]:
cols=[
    'CO(GT)', 'C6H6(GT)',
    'T', 'RH', 'AH'
]

for col in cols:
    df[col] = df[col].str.replace(',','.', regex=False).astype(float)
    print(f"Column {col} has been changed.")

In [ ]:
#Column Date
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y', errors='coerce') # Ubah kolom 'Date' menjadi format datetime
df['Date'] = pd.to_datetime(df['Date'])

#Column Time
df['Time'] = pd.to_datetime(df['Time'], format='%H.%M.%S', errors='coerce') # Ubah format 'Time' dari string ke datetime
df['Time'] = df['Time'].dt.strftime('%H:%M') # Ubah dalam format "HH:MM"

In [ ]:
df.sample(5)

### Sneak Peak Correlation Matrix

In [ ]:
df.dtypes

In [ ]:
# DROP index terlebih dahulu
df.reset_index(drop=True, inplace=True)

In [ ]:
numeric_df = df.select_dtypes(include=['int32', 'int64', 'float64'])

In [ ]:
# Buat heatmap korelasi
plt.figure(figsize=(10, 8))
sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Numeric Columns')
plt.tight_layout()
# plt.savefig("correlation_heatmap.png")
plt.show()

In [ ]:
#Next
numeric_df.corr()

### Outlier Analysis

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df_model['T'], bins=100, kde=True)  # bins= menentukan jumlah interval
plt.title('Distribusi Temperature (C)', fontsize=16)
plt.xlabel('Temp', fontsize=12)
plt.ylabel('Freq', fontsize=12)
plt.show()

In [ ]:
### drop Value -200
df = df[df['T'] != -200]

In [ ]:
df.shape

#### Box Plot

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x=df['T'])
plt.title('Temperatue (C)', fontsize=16)
plt.xlabel('Degree', fontsize=12)
plt.show()

#### Z-Score

In [ ]:
from scipy.stats import zscore

In [ ]:
df['Z_Temp'] = zscore(df['T'])

In [ ]:
df[['T','Z_Temp']].head()

In [ ]:
# Menentukan outlier (nilai Z-Score > 3 atau < -3)
outliers = df[(df['Z_Temp'] <= -3) | (df['Z_Temp'] >= 3)]
print("Outliers Detected:\n", outliers)

In [ ]:
# Visualisasi menggunakan Scatter Plot
plt.figure(figsize=(15, 5))
plt.scatter(df.index, df['Z_Temp'], label='Data', color='blue')

# Menambahkan outlier ke plot
plt.scatter(outliers.index, outliers['Z_Temp'], color='red', label='Outlier', s=100)
plt.title('Scatter Plot Temp with Outlier')
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()
plt.show()

### Analyze Value -200

In [ ]:
df.columns

In [ ]:
col1 = [
    'CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)',
    'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)',
    'PT08.S5(O3)', 'T', 'RH', 'AH'
]

for cols in col1:
    print(f"Columns {cols} :")
    print(df[df[cols] == -200])

In [ ]:
df[(df['NO2(GT)'] == -200) & (df['NOx(GT)'] == -200) & (df['NMHC(GT)'] == -200) & (df['CO(GT)'] == -200)]

In [ ]:
# DROP Column NMHC(GT) cause the value cannot compatible with Model
df[df['NO2(GT)'] == -200].value_counts()

In [ ]:
df.sample(5)

In [ ]:
df['NMHC(GT)'] == -200

### Copy Model & Create New DF

In [ ]:
df_model = df.copy()

In [ ]:
df_model.sample(5)

In [ ]:
df_model.dtypes

In [ ]:
df_model = df_model.drop('NMHC(GT)', axis=1)

In [ ]:
df_model.columns

In [ ]:
# DROP index terlebih dahulu
df_model.reset_index(drop=True, inplace=True)

In [ ]:
df_model.sample(8)

In [ ]:
col_mod = [
    'CO(GT)', 'NOx(GT)', 'NO2(GT)'
]

# Mengganti nilai -200 dengan rata-rata nilai sebelumnya dan sesudahnya
for cols in col_mod:
    for i in range(1, len(df_model) - 1):
        if df_model.iloc[i][cols] == -200:  # Menggunakan iloc untuk akses berdasarkan posisi
            prev_value = df_model.iloc[i - 1][cols]
            next_value = df_model.iloc[i + 1][cols]
            df_model.iloc[i, df_model.columns.get_loc(cols)] = (prev_value + next_value) / 2

In [ ]:
num_df= df.select_dtypes(['float64'])

In [ ]:
# Buat heatmap korelasi
plt.figure(figsize=(10, 8))
sns.heatmap(num_df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Numeric Columns')
plt.tight_layout()
# plt.savefig("correlation_heatmap.png")
plt.show()

In [ ]:
df_model[(df_model['NO2(GT)'] == -200) & (df_model['NOx(GT)'] == -200) & (df_model['CO(GT)'] == -200)]

In [ ]:
# df_model[df_model['CO(GT)'] == -200]
# df_model.head(10)
df_model.loc[600:700]

we will try to handling null value, symbolize by -200.

In [ ]:
# Daftar kolom yang ingin diperbaiki
col_mod = ['CO(GT)', 'NOx(GT)', 'NO2(GT)']

# Ganti -200 dengan nilai NaN agar bisa diinterpolasi
df_model[col_mod] = df_model[col_mod].replace(-200, pd.NA)

# Konversi kolom ke tipe numerik (jika belum)
for col in col_mod:
    df_model[col] = pd.to_numeric(df_model[col], errors='coerce')

# Lakukan interpolasi linier (berdasarkan index, karena datanya urut waktu)
df_model[col_mod] = (df_model[col_mod].interpolate(method='linear')).round(3)

# Jika masih ada NaN di awal atau akhir, bisa gunakan fill
# df_model[col_mod] = df_model[col_mod].fillna(method='bfill').fillna(method='ffill') #past Code
df_model[col_mod] = df_model[col_mod].bfill().ffill()

#last method fill value with mean
df_model[col_mod] = df_model[col_mod].fillna(value=df_model[col_mod].mean())

In [ ]:
df_model.dtypes

#### Corr after Data Cleaning

In [ ]:
nums_df= df_model.select_dtypes(['float64'])

In [ ]:
# Buat heatmap korelasi
plt.figure(figsize=(10, 8))
sns.heatmap(nums_df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Numeric Columns')
plt.tight_layout()
plt.show()

In [ ]:
df_model[(df_model['NO2(GT)'] == -200) & (df_model['NOx(GT)'] == -200) & (df_model['CO(GT)'] == -200)]

### Outlier Analysis after Cleaning

In [ ]:
df_model = df_model[df_model['T'] != -200]

In [ ]:
df_model.shape

In [ ]:
df_model.columns

In [ ]:
col_clean = ['CO(GT)', 'PT08.S1(CO)', 'C6H6(GT)', 'PT08.S2(NMHC)', 
             'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)', 'PT08.S5(O3)',
             'T', 'RH', 'AH']

for col in col_clean:
    plt.figure(figsize=(8, 4))
    sns.histplot(df_model[col], bins=100, kde=True)
    plt.title(f'Distribusi {col}', fontsize=16)
    plt.xlabel(col, fontsize=12)
    plt.ylabel('Frekuensi', fontsize=12)
    plt.show()

In [ ]:
### I think have a outlier in CO Distribution
df_model['CO(GT)'].value_counts()